In [6]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import accuracy_score

newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')


In [7]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english', min_df=3)),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('clf', LinearSVC(dual='auto', max_iter=10000)),
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__C': [0.1, 1, 10],
}

gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, verbose=1) # ma wbuowany cross-validation

gs_clf.fit(newsgroups_train.data, newsgroups_train.target)

# # Wynik z walidacji krzyżowej (to jest to 'x' do oceny zadania)
# cv_score = gs_clf.best_score_ * 100
# print(f"\n--- WYNIKI WALIDACJI KRZYŻOWEJ (Do oceny) ---")
# print(f"Najlepsze parametry: {gs_clf.best_params_}")
# print(f"Średnia dokładność CV (x): {cv_score:.2f}%")
# print(f"Punkty za zadanie (x - 85): {cv_score - 85:.2f}")

predicted = gs_clf.predict(newsgroups_test.data)
test_acc = accuracy_score(newsgroups_test.target, predicted) * 100

print(f"Dokładność na zbiorze testowym: {test_acc:.2f}%")

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Dokładność na zbiorze testowym: 86.54%


In [8]:
print(f"Najlepsze parametry: {gs_clf.best_params_}")


Najlepsze parametry: {'clf__C': 1, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# Pobieranie danych
# remove=('headers', 'footers', 'quotes') - opcjonalnie, czasem usunięcie nagłówków
# utrudnia zadanie, ale daje bardziej realistyczny model NLP. W zadaniu akademickim
# zazwyczaj trenujemy na "brudnym" zestawie, chyba że polecenie mówi inaczej.
newsgroups_train = fetch_20newsgroups(subset='train', remove=(), random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test', remove=(), random_state=42)

# Pipeline z SGDClassifier (szybszy i elastyczniejszy niż LinearSVC)
pipeline = Pipeline([
    ('vect', CountVectorizer(strip_accents='unicode', stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3, early_stopping=True)) 
])

# Siatka parametrów - szukamy "sweet spot"
parameters = {
    # N-gramy: klucz do kontekstu. (1,2) prawie zawsze wygrywa z (1,1)
    'vect__ngram_range': [(1, 1), (1, 2)],
    
    # Ograniczenie słownika - czasem wyrzucenie rzadkich słów pomaga (szum)
    'vect__min_df': [1, 2, 3], 
    'vect__max_df': [0.5, 0.75, 1.0],

    # Użycie IDF
    'tfidf__use_idf': [True],
    'tfidf__sublinear_tf': [True, False], # Logarytmiczne skalowanie częstości

    # Parametry modelu
    'clf__alpha': [1e-3, 1e-4, 5e-5, 1e-5], # Regularyzacja
    'clf__penalty': ['l2', 'elasticnet'],   # Rodzaj kary
    'clf__loss': ['hinge', 'log_loss'],     # hinge = SVM, log_loss = Regresja Logistyczna
}

# GridSearch
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)

print("Trwa trenowanie GridSearch...")
grid_search.fit(newsgroups_train.data, newsgroups_train.target)

print(f"\nNajlepszy wynik walidacji krzyżowej (k-fold): {grid_search.best_score_ * 100:.2f}%")
print("Najlepsze parametry:", grid_search.best_params_)

# Test na zbiorze testowym
predicted = grid_search.predict(newsgroups_test.data)
accuracy = np.mean(predicted == newsgroups_test.target) * 100
print(f"Dokładność na zbiorze testowym: {accuracy:.2f}%")

# Obliczenie punktów wg wzoru z zadania
points = grid_search.best_score_ * 100 - 85
print(f"Szacowane punkty za zadanie (x - 85): {points:.2f}")

Trwa trenowanie GridSearch...
Fitting 5 folds for each of 576 candidates, totalling 2880 fits
